In [0]:
from pyspark.sql.functions import col, year, month, to_date

In [0]:
reviews_with_time = spark.table("workspace.default.silverreview") \
    .withColumn("date_parsed", to_date(col("date"))) \
    .withColumn("year", year(col("date_parsed"))) \
    .withColumn("month", month(col("date_parsed")))

In [0]:
silverbusiness = spark.table("workspace.default.silverbusiness") \
    .drop("_ingest_ts") \
    .withColumnRenamed("stars", "business_stars") \
    .withColumnRenamed("review_count", "business_review_count") \
    .withColumnRenamed("name", "business_name")

silveruser = spark.table("workspace.default.silveruser") \
    .drop("_ingest_ts") \
    .withColumnRenamed("cool", "user_cool") \
    .withColumnRenamed("funny", "user_funny") \
    .withColumnRenamed("useful", "user_useful") \
    .withColumnRenamed("review_count", "user_review_count") \
    .withColumnRenamed("name", "user_name")

In [0]:
gold_master_reviews = (
    reviews_with_time.alias("r")
    .join(silverbusiness.alias("b"), "business_id", "left") 
    .join(silveruser.alias("u"), "user_id", "left")
)

In [0]:
print(gold_master_reviews.columns)

In [0]:
gold_master_reviews.display

In [0]:
gold_master_reviews.write.format("delta").mode("overwrite") \
    .saveAsTable("workspace.default.gold_review_master1")